In [44]:
# 加载模型
from transformers import AutoTokenizer, T5ForConditionalGeneration
pretrained = "../t5_corrector_v2"
tokenizer = AutoTokenizer.from_pretrained(pretrained)
model = T5ForConditionalGeneration.from_pretrained(pretrained)

In [45]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def correct(text, max_length):
    model_inputs = tokenizer(text, 
                                max_length=max_length, 
                                truncation=True, 
                                return_tensors="pt").to(device)
    output = model.generate(**model_inputs, 
                              num_beams=5,
                              no_repeat_ngram_size=4,
                              do_sample=True, 
                              early_stopping=True,
                              max_length=max_length,
                              return_dict_in_generate=True,
                              output_scores=True)
    pred_output = tokenizer.batch_decode(output.sequences, skip_special_tokens=True)[0]
    return pred_output

text = "下个星期，我跟我朋唷打算去法国玩儿。"
correction = correct(text, max_length=32)
print(correction)

下个星期,我跟我朋唷打算去法国玩儿。


In [48]:
from pycorrector.t5.t5_corrector import T5Corrector
nlp1 = T5Corrector("../t5_corrector_v2").batch_t5_correct
nlp2 = T5Corrector("../mengzi_t5_base_correction").batch_t5_correct

2023-11-08 21:03:54.693 | DEBUG    | pycorrector.t5.t5_corrector:__init__:59 - Use device: cuda
2023-11-08 21:03:54.694 | DEBUG    | pycorrector.t5.t5_corrector:__init__:60 - Loaded t5 correction model: ../t5_corrector_v2, spend: 2.442 s.
2023-11-08 21:03:57.304 | DEBUG    | pycorrector.t5.t5_corrector:__init__:59 - Use device: cuda
2023-11-08 21:03:57.305 | DEBUG    | pycorrector.t5.t5_corrector:__init__:60 - Loaded t5 correction model: ../mengzi_t5_base_correction, spend: 2.610 s.


In [1]:
import random
from pypinyin import lazy_pinyin
def mask_text(text: str, prob:float=0.1, momentum:float=0.9):
    masked_text = ""
    factor = 0
    for c in text:
        if random.random() < prob*pow(momentum, factor) and ('\u4e00' <= c <= '\u9fff'):
            masked_text += "[MASK]"
            factor += 1
        else:
            masked_text += c
            factor = 0
    return masked_text

def text2pinyin(text):
    pinyin_text = ""
    for c in text:
        if '\u4e00' <= c <= '\u9fff':
            pinyin_text += lazy_pinyin(c)[0]
        else:
            pinyin_text += "[OTHER]"
        pinyin_text += " "
    return pinyin_text.strip()

In [3]:
pinyin_map = {}
with open('../chinese-bert-wwm/pinyin_vocab.txt') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    pinyin_map[line.strip('\n')] = i

In [3]:
import json
from tqdm import tqdm
with open("../SIGHAN/train.json", encoding='utf-8') as f:
    train_data = json.load(f)
with open("../SIGHAN/train.txt", 'a', encoding='utf-8') as f:
    for data in tqdm(train_data):
        f.write(f"{mask_text(data['correct_text'], prob=0.05)}\t{data['correct_text']}\n")

100%|██████████| 251835/251835 [00:03<00:00, 79988.52it/s]


In [5]:
with open("../chinese-bert-wwm/vocab.txt", encoding="utf-8") as  f:
    vocabs = f.readlines()
pinyin_idx = {}
with open("../chinese-bert-wwm/pinyin_vocab.txt", encoding="utf-8") as f:
    pinyin_vocab = f.readlines()
    for i, pinyin in enumerate(pinyin_vocab):
        pinyin_idx[pinyin.strip("\n")] = i

In [13]:
from pypinyin import lazy_pinyin
lazy_pinyin("wo")

['wo']

In [19]:
import json
from pypinyin import lazy_pinyin
pinyin_map = {}
with open("../chinese-bert-wwm/pinyin_map.json", "w") as f:
    for i, vocab in enumerate(vocabs):
        if pinyin_idx.get(vocab.strip("\n")) is not None and vocab[0] == '[':
            pinyin_map[i] = pinyin_idx[vocab.strip("\n")]
            print(vocab)
        elif '\u4e00' <= vocab.strip("\n") <= '\u9fff':
            pinyin_map[i] = pinyin_idx[lazy_pinyin(vocab.strip("\n"))[0]]
        else:
            pinyin_map[i] = pinyin_idx["[OTHER]"]
    json.dump(pinyin_map, f, indent=4)

[PAD]

[CLS]

[SEP]

[MASK]



In [18]:
pinyin_idx.get(vocabs[0].strip("\n"))

0

In [1]:
import torch
from transformers import BertForMaskedLM, BertTokenizer
model = BertForMaskedLM.from_pretrained('../chinese-bert-wwm/')

/home/daiyuxin/anaconda3/envs/had/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ../chinese-bert-wwm/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
tokenizer = BertTokenizer.from_pretrained("../chinese-bert-wwm/")

In [85]:
text = "目前这次事件的细节还不清楚，[MASK]亡人数也未确定。"
target = "我下个礼拜已[MASK]搬家到新的地址了，在这里我觉得那么方便，离学校不太远，去超[MASK]市场很近。"
# tokens = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    output = model(**tokenizer([text, target], return_tensors='pt', max_length=64, padding=True, truncation=True))

In [87]:
masked_lm_logits = output.logits
predicted_token_ids = torch.argmax(masked_lm_logits, dim=-1)

# 将预测的标记转换回文本
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[1])

# 输出最佳预测
print("Predicted tokens:", "".join(predicted_tokens[1:-1]))

Predicted tokens: 我下个礼拜已经搬家到新的地址了，在这里我觉得那么方便，离学校不太远，去超级市场很近。


In [88]:
with open("../chinese-bert-wwm/pinyin_vocab.txt") as f:
    lines = f.readlines()
with open("../chinese-bert-wwm/pinyin_vocab.txt", 'w') as f:
    f.writelines([line for i, line in enumerate(lines) if i%5 == 0])

In [94]:
from pypinyin import lazy_pinyin

In [103]:
lazy_pinyin("你还欠钱还乡？")

['ni', 'hai', 'qian', 'qian', 'huan', 'xiang', '？']